In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch.utils.data
from sklearn.model_selection import train_test_split
from beam_utils import ULA_DFT_codebook,UPA_DFT_codebook,calc_beam_pattern,pow_2_dB,dB_2_pow
import torch
from DL_utils_MISO import BF_Autoencoder
import DeepMIMO
from os.path import exists

In [5]:
scenario = 'Boston5G_28'
activated_BS = 1
random_UE_rotation = False
array_type = 'ULA'

tx_power_dBm = 40
noise_PSD_dB = -161
BW = 50e6
noise_power_dBm = noise_PSD_dB + 10*np.log10(BW) # dBm
measurement_gain = 32.0
measurement_gain_dB = 10*np.log10(measurement_gain)
IA_threshold = -5
measurement_noise_power = 10**((noise_power_dBm-tx_power_dBm)/10)/measurement_gain
h_NMSE_dB = -np.inf
dataset_split_seed = 7
BS_array_shape = np.array([1,64,1])

In [6]:
scenario == 'Boston5G_28'
ue_row_first = 1
ue_row_last = 1622

parameters = DeepMIMO.default_params()
parameters['dataset_folder'] = 'D:/Github Repositories/DeepMIMO-codes/DeepMIMOv2/Raytracing_scenarios'

parameters['scenario'] = scenario
parameters['num_paths'] = 15
parameters['active_BS'] = [activated_BS]
parameters['user_row_first'] = ue_row_first
parameters['user_row_last'] = ue_row_last
parameters['row_subsampling'] = 1
parameters['user_subsampling'] = 1

parameters['bs_antenna']['shape'] = BS_array_shape
parameters['bs_antenna']['spacing'] = 0.5
parameters['bs_antenna']['radiation_pattern'] = 'isotropic'

parameters['ue_antenna']['shape'] = np.array([1, 1, 1])
parameters['ue_antenna']['spacing'] = 0.5
parameters['ue_antenna']['radiation_pattern'] = 'isotropic'

parameters['enable_BS2BS'] = 0

parameters['OFDM_channels'] = 1 # Frequency (OFDM) or time domain channels
parameters['OFDM']['subcarriers'] = 512
parameters['OFDM']['subcarriers_limit'] = 1
parameters['OFDM']['subcarriers_sampling'] = 1
parameters['OFDM']['bandwidth'] = BW/1e9
parameters['OFDM']['RX_filter'] = 0

dataset = DeepMIMO.generate_data(parameters)

dataset_savefname = './Data/{}_{}x{}x{}.npy'.format(scenario,
                                                BS_array_shape[0],
                                                BS_array_shape[1],
                                                BS_array_shape[2])

if exists(dataset_savefname):
    print('Loading dataset...')
    h = np.load(dataset_savefname,allow_pickle=True)
else:
    print('Generating dataset...')
    dataset = DeepMIMO.generate_data(parameters)
    h = dataset[0]['user']['channel'].squeeze(axis=-1)
    valid_ue_idc = np.array([ue_i for (ue_i,ue_h) in enumerate(h) if not (ue_h==0).all()])
    print('Keep {} out of {} UEs that have valid paths.'.format(len(valid_ue_idc),h.shape[0]))
    h = h[valid_ue_idc]
    np.save(dataset_savefname,h,allow_pickle=True)


Basestation 1

UE-BS Channels


Reading ray-tracing:   0%|          | 605/965090 [00:00<13:56, 1152.71it/s]


KeyboardInterrupt: 